# DAY 8: Model Interpretation and data manipulation

### Machine Learning and Computational Statistics (DSC6232)

#### Instructors: Weiwei Pan, Melanie Fernandez, Pavlos Protopapas

#### Due: August 3rd, 2:00 pm Kigali Time

**First name**: _________________________________________________________


**Last name**: _____________

## Learning Goals:

1. encode non-numerical data
2. preprocess data: scaling covariates, outlier removal and data imputation
3. model interpretation for down-stream application

### Load necessary libraries

In [ ]:
# as usual, let us load all the necessary libraries
import numpy as np  # numerical computation with arrays
import pandas as pd # library to manipulate datasets using dataframes
import scipy as sp  # statistical library

# below sklearn libraries for different models
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# plot 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

### We include auxiliary functions here that we will need to use later 

**No need to read in details!**

In [ ]:
# function to generate random splits
def split_data(X, y ,seed=29):
    ''' Function to split  randomly your dataset in train and test '''
    N,D = X.shape
    test_size=0.4
    np.random.seed(seed)
    permuted_idxs = np.random.permutation(N)
    N_train = int(np.floor(N*(1-test_size)))
    train_idxs = permuted_idxs[:N_train]
    test_idxs = permuted_idxs[N_train:]
    X_train = X[train_idxs,:]
    X_test = X[test_idxs,:]
    y_train = y[train_idxs]
    y_test = y[test_idxs]
    return X_train, X_test, y_train, y_test

# function to print feature importances (size of coefficients)
def print_sorted_feat_importance(weights, colnames):
    feature_weights = pd.Series(weights, index=colnames)
    feature_weights_sorted = feature_weights.apply(np.abs).sort_values(ascending=False)

    for name in feature_weights_sorted.index:
         print('{:23s}: {:.4f}'.format(name, feature_weights[name]))
            
# function to plot confidence intervals
def plot_confidence_intervals(bootstrap_weights, ax):
    rows = bootstrap_weights.shape[0]
    columns = bootstrap_weights.shape[1]
    bootstrap_weights = bootstrap_weights.reshape((rows, columns))
    ax.set_title("Variation in weight across variations in train set")
    ax.boxplot(bootstrap_weights)
    ax.set_ylabel('Weight values')
    ax.set_xlabel('Features')
    ax.set_xticklabels(X_colnames, rotation=90)
    return ax

# function to plot predictive intervals
def plot_predictive_intervals(bootstrap_predictions, labels, ax):
    rows = bootstrap_predictions.shape[0]
    columns = bootstrap_predictions.shape[1]
    bootstrap_predictions = bootstrap_predictions.reshape((rows, columns))
    ax.set_title("Variation in prediction across variations in train set")
    ax.boxplot(bootstrap_predictions)
    ax.set_ylabel('Predicted Sales')
    ax.set_xlabel('Companies')
    ax.set_xticklabels(labels, rotation=90)
    return ax

def get_data(messy_data=True):

    # load the data using pandas
    X_df = pd.read_csv('https://raw.githubusercontent.com/onefishy/Rwanda-course-2020/master/In-class/X.csv')
    y_df = pd.read_csv('https://raw.githubusercontent.com/onefishy/Rwanda-course-2020/master/In-class/y.csv')

    # change $ to cents
    X_df['bus_stops ($)'] *= 100.0
    X_df.rename(columns={'bus_stops ($)': 'bus_stops (cents)'}, inplace=True)

    # add id
    X_df['company id number'] = np.arange(X_df.shape[0])

    # add country
    r = X_df['TV ($)']
    r += np.random.normal(scale=2.0, size=r.shape)
    X_df['Country'] = r
    X_df.loc[r < 25.0, 'Country'] = 'Mexico'
    X_df.loc[(r > 25.0) & (r < 160.0), 'Country'] = 'USA'
    X_df.loc[r > 160.0, 'Country'] = 'Rwanda'

    X_df['Nationality'] = r
    X_df.loc[r < 25.0, 'Nationality'] = 'Mexican'
    X_df.loc[(r > 25.0) & (r < 160.0), 'Nationality'] = 'American'
    X_df.loc[r > 160.0, 'Nationality'] = 'Rwandan'


    if messy_data:    
        # create outliers
        p_outliers = 0.1
        outlier_idx = X_df.sample(frac=p_outliers, replace=True).index
        X_outliers = X_df.iloc[outlier_idx].copy()
        X_outliers[['electricity (mega W)', 'water (10^3 m3)']] += 30.0
        X_df = pd.concat([X_df, X_outliers], ignore_index=True)
        y_df = pd.concat([y_df, y_df.iloc[outlier_idx]], ignore_index=True)

        # add missing values
        p_missing_per_columnn = 0.01
        for c in X_df.columns:
            missing_idx = X_df.sample(frac=p_missing_per_columnn, replace=False).index
            X_missing = X_df.iloc[missing_idx].copy()
            X_missing[c] = np.NaN

            X_df = pd.concat([X_df, X_missing], ignore_index=True)
            y_df = pd.concat([y_df, y_df.iloc[missing_idx]], ignore_index=True)

    new_index = np.random.permutation(X_df.index)
    X_df = X_df.reindex(new_index)
    y_df = y_df.reindex(new_index)

    return X_df, y_df

---

## Predicting total sales from marketing strategy

In this exercise, you are asked to build a machine learning model to predict the total sales of a company based on their marketing strategy (how much money they invest in advertisement and in which venues they choose to advertise) as well as a few additional features of the company.

The goal is not only to predict how much sales a company should expect, the goal is to also make a recommendation to the company on how they should adjust their marketing strategy or internal operations (e.g. you might recommend, based on your model, that the company invests more in social media marketing).

#### Load the dataset and examine it

In [ ]:
# load the data using pandas
X_df, y_df = get_data()

In [ ]:
# print the first rows of the marketing strategies data
X_df.head()

,electricity (mega W),water (10^3 m3),bus_stops (cents),sport_events ($),social_media ($),TV ($),radio ($),newspaper ($),company id number,Country,Nationality
195,0.211000,1.452358,2137.583559,31.573711,42.415801,41.684291,3.7,13.8,195.0,USA,American
210,32.754776,32.524236,97.807237,20.267720,32.773621,167.445140,42.0,3.6,193.0,Rwanda,Rwandan
157,1.582856,1.440923,1198.350520,38.072263,6.668631,150.494113,1.3,24.3,157.0,USA,American
174,0.575027,0.454591,3652.214898,18.817164,14.940963,220.435758,3.4,13.1,174.0,Rwanda,Rwandan
172,1.197656,0.595153,4676.287407,16.815753,30.320105,18.549877,20.1,17.0,172.0,Mexico,Mexican


**Exercise 1:** Are there any covariates we need to encode? If so, which ones?

In [ ]:
# print the first rows of the outcomes (total sales)
y_df.head()

,sales (10^3 units)
195,7.6
210,19.6
157,10.1
174,11.5
172,7.6


In [ ]:
# get a summary of how many rows in the dataset and how many missing values is in each column
# the 'Non-Null Count' counts the number of values in each column that is not missing
X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 195 to 188
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   electricity (mega W)  240 non-null    float64
 1   water (10^3 m3)       239 non-null    float64
 2   bus_stops (cents)     240 non-null    float64
 3   sport_events ($)      240 non-null    float64
 4   social_media ($)      240 non-null    float64
 5   TV ($)                240 non-null    float64
 6   radio ($)             240 non-null    float64
 7   newspaper ($)         240 non-null    float64
 8   company id number     240 non-null    float64
 9   Country               240 non-null    object 
 10  Nationality           240 non-null    object 
dtypes: float64(9), object(2)
memory usage: 22.7+ KB


**Exercise 2:** How many companies are contained in this dataset? For each covariate or column in the dataset, how many values are missing?

In [ ]:
# print a summary of the values for each covariate in the dataset
X_df.describe()

NameError: ignored

**Exercise 3:** Do we need to scale the covariates in this dataset? Why or why not?

---
## Pre-process the data

We apply one-hot encoding to  categorical covariates, scale the values in the covariates, impute missingness and remove outliers.

#### One-hot encode categorical variables

In [ ]:
# one hot encode the covariate 'Country'
one_hot_country = pd.get_dummies(X_df['Country'], prefix='Country')

# use pd.concat to join the new columns with your original dataframe
X_df = pd.concat([X_df, one_hot_country], axis=1)

# now drop the original 'country' column (you don't need it anymore)
X_df.drop(['Country'], axis=1, inplace=True)

# print the resultant dataframe
X_df.head()

,electricity (mega W),water (10^3 m3),bus_stops (cents),sport_events ($),social_media ($),TV ($),radio ($),newspaper ($),company id number,Nationality,Country_Mexico,Country_Rwanda,Country_USA
195,0.211000,1.452358,2137.583559,31.573711,42.415801,41.684291,3.7,13.8,195.0,American,0,0,1
210,32.754776,32.524236,97.807237,20.267720,32.773621,167.445140,42.0,3.6,193.0,Rwandan,0,1,0
157,1.582856,1.440923,1198.350520,38.072263,6.668631,150.494113,1.3,24.3,157.0,American,0,0,1
174,0.575027,0.454591,3652.214898,18.817164,14.940963,220.435758,3.4,13.1,174.0,Rwandan,0,1,0
172,1.197656,0.595153,4676.287407,16.815753,30.320105,18.549877,20.1,17.0,172.0,Mexican,1,0,0


**Exercise 4:** Encode the remaining categorical variables.

In [ ]:
# Step 1: one hot encode the categorical covariate

# Step 2: use pd.concat to join the new columns with your original dataframe

# Stpe 3: now drop the original 'country' column (you don't need it anymore)

# Step 4: print the resultant dataframe

**Exercise 5:** Inspect your DataFrame, are there any covariates that are irrelevant for prediction $y$? Drop unimportant covariates. 

***Hint:*** does the id number of the compny help predict total revenue?

Are there any covariates that are redundant, i.e. they contain the exact information as other covariates? Drop redundant covariates.

In [ ]:
# Step 1: Drop unimportant covariates
# Hint: look up the documentation for pandas' .drop() function
# COMPLETE
# X_df.drop(...)

# Step 2: Drop ALL redundant covariates
# Hint: look up the documentation for pandas' .drop() function
# COMPLETE
# X_df.drop(...)

#### Impute missing values

**Exercise 5:** Impute the missing values in each column using a k-Nearest Neighbor model.

In [ ]:
# Step 1: define an knn imputer using KNNImputer
# Hint: look up the documentation for KNNImputer
# COMPLETE
# imputer = ...

# Step 2: impute the missing data
# X_df = pd.DataFrame(imputer.fit_transform(X_df), columns=X_df.columns, index=X_df.index)

#### Remove outliers

**Exercise 6:** Remove the outlier values in each column of the dataset.

In [ ]:
low = .05
high = .95

# Step 1: compute 5% percentile and the 95% percentile of each column in the dataset
quantile_df = X_df.quantile([low, high])
quantile_df

,electricity (mega W),water (10^3 m3),bus_stops (cents),sport_events ($),social_media ($),TV ($),radio ($),newspaper ($),company id number,Country_Mexico,Country_Rwanda,Country_USA
0.05,0.004004,0.008829,0.059869,0.038170,0.070265,0.02838,0.040222,0.029024,0.050000,0.0,0.0,0.0
0.95,0.951807,0.953139,0.952286,0.975905,0.969771,0.94567,0.948286,0.628848,0.959799,1.0,1.0,1.0


In [ ]:
# Step 2: remove ouliers in 'water (10^3 m3)'

# print the number of rows before outlier removal
print('Number of rows before outlier removal: {}'.format(X_df.shape[0]))
# select rows greater than the 5% and smaller than the 95% threshold
X_df = X_df[(X_df['water (10^3 m3)'] > quantile_df.loc[0.05, 'water (10^3 m3)']) & (X_df['water (10^3 m3)'] < quantile_df.loc[0.95, 'water (10^3 m3)'])]
# print the number of rows after outlier removal
print('Number of rows after outlier removal: {}'.format(X_df.shape[0]))

Number of rows before outlier removal: 242
Number of rows after outlier removal: 215


In [ ]:
# Step 3: perform outlier removal for all columns
# COMPLETE

#### Scale the data

**Exercise 7:** Scale the values of the covariates so that they all lie in the range of $[0, 1]$. 

In [ ]:
# Step 1: define a MinMax scalar that will transform the data values into values in (0, 1)
# Hint: Look up the documentation for MinMaxScaler
# COMPLETE
# scaler = ...
scalar = MinMaxScaler()

# Step 2: fit the MinMaxScaler using our data 
# Hint: use .fit()
# COMPLETE
X_values = X_df.values
# Step 3: scale the values in our dataset, X_values 
# Hint: use .transform()
# COMPLETE
# X_values_scaled = ...
# X_df = pd.DataFrame(X_values_scaled, columns=X_df.columns)

---
## Model the data

Now that we have preprocessed the data, we are ready to build models to predict the label $y$ using the covariates $X$.


#### Split the data into training and testing

In [ ]:
# transform the data into numpy arrays
X = X_df.values
y = y_df.values

# store the names of the columns
X_colnames = X_df.columns.values
y_colnames = y_df.columns.values

In [ ]:
# split data into train and test
X_train, X_test, y_train, y_test = split_data(X,y)
# name the companies in the test set
indices = ['company %d' % d for d in range(1,X_test.shape[0]+1)]
X_test_df = pd.DataFrame(X_test, columns=X_colnames, index=indices)

#### Train a linear regression model

**Exercise 8:** Train a linear regression model to predict the total sales $y$ from the marketing strategy data $X$. Evaluate the train and test MSE of you model. 

How is the fit of your model? Would you feel comfortable using this model to make real marketing decisions? Why or why not?

In [ ]:
# Step 1: create a sklearn linear regression model
# COMPLETE
# linear_regressor = ...

# Step 2: fit the linear regression model on the data
# COMPLETE

In [ ]:
# Step 3: compute MSE on train dataset
# COMPLETE
# score_train = ...

# Step 4: compute MSE on test dataset
# COMPLETE
# score_test = ...

# print('MSE on train set: %.2f' % score_train)
# print('MSE on test set: %.2f' % score_test)

**Exercise 9:** There are three subgroups of companies in your dataset, "USA", "Rwanda" and "Mexico". Evaluate the test MSE on each subgroup of companies in your dataset. 

In [ ]:
# Step 1: select all companies from Rwanda
# country_is_rwanda = (X_test_df['Country_Rwanda'] == 1)

# Step 2: compute the test MSE for these companies
# score_test = mean_squared_error(y_test[country_is_rwanda], linear_regressor.predict(X_test_df[country_is_rwanda]))
# print('MSE on test set (Rwanda): %.2f' % score_test)

# Step 3: evaluate the train MSE for companies from the USA
# COMPLETE

# Step 4: evaluate the train MSE for companies from Mexico
# COMPLETE

**Exercise 10:** For which group of companies does your model do the best? For which group of companies does your model do the worst? Explain why your model does well on some groups but not others.

***Hint:*** Count the number of companies of each nationality.

In [ ]:
# count the number of data rows that has a 1 in the 'Country_Rwanda' column
count_rwanda = X_df['Country_Rwanda'].sum()
print('Number of entries from Rwanda', count_rwanda)

Number of entries from Rwanda 117


---

## Interpret the model

Often, we are not just interested in obtaining a prediction from a model, we are also interested in why the model has made that prediction.

For a linear regression model, $y = w_0 + w_1x_1 + w_2x_2 + \ldots + w_Dx_D$, looking at the coefficients can help us determine which feature was an important factor in the prediction.

The coefficients of `sklearn`'s linear regression model is stored in the model's `.coef_` parameter.

**Exercise 11:** Based on your model coefficients, which feature do you think has the most effect on the total sales? Can you hypothesize why?

In [ ]:
# Step 1: get the coefficients for the linear regression model
# weights = linear_regressor.coef_[0]

# Step 2: print the coefficients in descending order (by absolute value)
# print_sorted_feat_importance(weights, X_colnames)

#### Constructing Confident Intervals for Regression Coefficients

Before we start making marketing recommendations based on the regression coefficients, let's determine if these coefficients are significant. That is, are the coefficients describing a rule that is particular only to our training data and cannot generalize to new data?

To do this, we change the training data slightly (by bootstrapping) and see how much our regression coefficients change.

**Exercise 12:** Based on the following box plot of the regression coefficient for models fitted on 5 bootstrapped training data sets, which feature do you think has the most effect on the total sales? Is this answer different from your answer in Exercise 1?

In [ ]:
# Step 1: Uncomment and run
# ### we fit 5 regression models on 5 bootstrapped samples of our training data
# # number of bootstrap samples to make
# n_bootstrap = 5
# # a list to store the weights of each regression model
# bootstrap_weights = []
# for n in range(n_bootstrap):
#     # create new training data
#     X_train, _, y_train, _ = split_data(X, y, seed=n)
#     # train linear regressor model
#     linear_regressor.fit(X_train,y_train)
#     # save the regression weights
#     bootstrap_weights.append(linear_regressor.coef_[0])
    
# bootstrap_weights = np.array(bootstrap_weights)

# # make a box plot for the regression weights
# fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# plot_confidence_intervals(bootstrap_weights, ax)
# plt.show()

#### Constructing the Predictive Interval of a Regression Model

In test data, company 30 is **Coca-cola** and company 29 is **Inyange**.

**Exercise 13:** Using your linear regression model predict which company will have a higher total sales based on their marketing strategy.

Construct the predictive interval for each predicted total sales. 

How confident are you in your predictions? Support your answers using concepts from lecture.

In [ ]:
X_test_df.head()

,electricity (mega W),water (10^3 m3),bus_stops (cents),sport_events ($),social_media ($),TV ($),radio ($),newspaper ($),company id number,Nationality,Country_Mexico,Country_Rwanda,Country_USA
company 1,0.18311,0.979121,1027.52,26.3546,6.12187,89.5255,0.3,23.2,107,American,0,0,1
company 2,30.6507,32.1584,2412.87,49.8748,26.5837,165.87,7.8,35.2,177,Rwandan,0,1,0
company 3,1.89494,1.00387,3896.79,48.5626,47.3148,208.835,49.6,37.7,58,Rwandan,0,1,0
company 4,2.81545,2.47702,74.0314,41.3383,22.4982,290.077,27.7,1.8,42,Rwandan,0,1,0
company 5,2.33241,1.13286,1326.8,0.226549,NaN,241.719,49,44.3,147,Rwandan,0,1,0


In [ ]:
# Step 1: predict on the test data
# COMPLETE
# y_predict = ...

# Step 2: get the 29th and 30th predictions
# COMPLETE
# y_pred_29 = ...
# y_pred_30 = ...

# print the total sales of company 29 and company 30
# print('Predicted sales for Coca-cola: %.2f (thousand units)' % y_pred_29)
# print('Predicted sales for Inyange: %.2f (thousand units)' % y_pred_30)

Construct the predictive interval for each predicted total sales. Based on these intervals, how confident are you in your predictions?

In [ ]:
# Step 1: Uncommnent and run
# ### we fit 1000 regression models on 1000 bootstrapped samples of our training data
# n_bootstrap = 1000
# # make a list to store the predicted sales
# bootstrap_y_pred = []
# for n in range(n_bootstrap):
#     # make new data
#     X_train, _, y_train, _ = split_data(X, y, seed=n)
#     # train linear regression model
#     linear_regressor.fit(X_train,y_train)
#     # predict on company 30 and 31
#     y_predict = linear_regressor.predict(X_test[29:31])
#     # add the prediction to the list
#     bootstrap_y_pred.append(y_predict)
# # convert the list into an array
# bootstrap_y_pred = np.array(bootstrap_y_pred)

# fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# plot_predictive_intervals(bootstrap_y_pred, ['Coca-cola', 'Inyange'], ax)
# plt.show()